In [72]:
using Optim, ForwardDiff, LinearAlgebra, Printf, Convex, ECOS

The n-variable Rosenbrock test function is

$$
f(x) = \sum_{i=1}^{N-1}[100(x_{i+1} - x_{i}^2)^2+(1-x_i)^2].
$$

Define the 3-variable Rosenbrock test function.

In [102]:
N = 3;
f3(x)= 2*x[1]^2 + 3*x[2]^2 + 4*x[3]^2 + 2*x[1]*x[2] -2*x[1]*x[3] -8*x[1] -4*x[2]-2*x[3]

f3 (generic function with 1 method)

In [103]:
g(x) = ForwardDiff.gradient(f3, x)

g (generic function with 1 method)

The linear constraint is $x_1+x_2+x3 = 1$ and $2x_1+x_2 = 0.5$.

In [104]:
AtA = [4 2 -2;2 6 0;-2 0 8]
L = norm(AtA,2)

11.489125293076057

let $C = \{x | Ax = b\}$. Projection on to C is $\text{proj}_{\mathcal C} (z) = z + A^\intercal(AA^\intercal)^{-1}(b-Az)$

In [105]:
proj(z) = max.(z,0)

proj (generic function with 1 method)

In [106]:
xk = zeros(3);
step = 1/L;
xkp = xk;
for i in 1:100
    xkp = xk - step * g(xk);
    xkp = proj(xkp)
    if rem(i,10) == 0
        @printf("iteration = %d, successive error = %f\n",i, norm(xk - xkp))
    end
    xk = xkp
end
@printf("\nThe minimizer is x = (%.3f,%.3f,%.3f) with optimal value = %.3f\n", xk[1], xk[2], xk[3],f3(xk))

iteration = 10, successive error = 0.058070
iteration = 20, successive error = 0.002295
iteration = 30, successive error = 0.000091
iteration = 40, successive error = 0.000004
iteration = 50, successive error = 0.000000
iteration = 60, successive error = 0.000000
iteration = 70, successive error = 0.000000
iteration = 80, successive error = 0.000000
iteration = 90, successive error = 0.000000
iteration = 100, successive error = 0.000000

The minimizer is x = (2.429,0.000,0.857) with optimal value = -10.571


In [114]:
chol = cholesky(AtA/2)
A = chol.U
b = (A')\([8;4;2]./2)

x = Variable(3)
problem = minimize(sumsquares(A*x - b) - norm(b,2)^2, [x>=0])
solve!(problem, ECOS.Optimizer(verbose = false))
@printf("\nThe minimizer is x = (%.3f,%.3f,%.3f) with optimal value = %.3f\n", x.value[1], x.value[2], x.value[3], problem.optval)


The minimizer is x = (2.429,0.000,0.857) with optimal value = -10.571


In [116]:
g(x.value)

3×1 Array{Float64,2}:
 -1.6013894210686885e-8
  0.8571428327137207
 -1.4521682167867311e-7